In [2]:
!pip install fiona
!pip install geopandas

     |████████████████████████████████| 14.7MB 1.3MB/s 
     |████████████████████████████████| 931kB 2.8MB/s 
     |████████████████████████████████| 10.9MB 17.1MB/s 


In [0]:
import pandas as pd
import numpy as np
import math

import geopandas
import json

import requests

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column

In [15]:
raw= requests.get("https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/ncov_cases2_v1/FeatureServer/2/query?where=1%3D1&outFields=*&outSR=4326&f=json")
raw_json = raw.json()
d = pd.DataFrame(raw_json["features"])
d.head()

,attributes,geometry
0,"{'OBJECTID': 1, 'Country_Region': 'Australia',...","{'x': 133.0000000000001, 'y': -24.999999999999..."
1,"{'OBJECTID': 2, 'Country_Region': 'Austria', '...","{'x': 14.550100000000045, 'y': 47.516200000000..."
2,"{'OBJECTID': 3, 'Country_Region': 'Canada', 'L...","{'x': -95.00099999999996, 'y': 60.00100000000003}"
3,"{'OBJECTID': 4, 'Country_Region': 'China', 'La...","{'x': 114.30550000000004, 'y': 30.592800000000..."
4,"{'OBJECTID': 5, 'Country_Region': 'Denmark', '...","{'x': 9.50180000000006, 'y': 56.263900000000035}"


In [47]:
data_list = d["attributes"].tolist()
df = pd.DataFrame(data_list)
df.set_index("OBJECTID")
df = df[["Country_Region", "Lat", "Long_", "Confirmed", "Deaths", "Recovered"]]
df.head()

,Country_Region,Lat,Long_,Confirmed,Deaths,Recovered
0,Australia,-25.0000,133.0000,7204,102,6619
1,Austria,47.5162,14.5501,16733,668,15596
2,Canada,60.0010,-95.0010,92479,7374,49214
3,China,30.5928,114.3055,84150,4638,79390
4,Denmark,56.2639,9.5018,11899,576,10610


In [17]:
df

,Country_Region,Lat,Long_,Confirmed,Deaths,Recovered
0,Australia,-25.000000,133.000000,7204,102,6619
1,Austria,47.516200,14.550100,16733,668,15596
2,Canada,60.001000,-95.001000,92479,7374,49214
3,China,30.592800,114.305500,84150,4638,79390
4,Denmark,56.263900,9.501800,11899,576,10610
...,...,...,...,...,...,...
183,West Bank and Gaza,31.952200,35.233200,449,3,372
184,Western Sahara,24.215500,-12.885800,9,1,6
185,Yemen,15.552727,48.516388,323,80,14
186,Zambia,-13.133897,27.849332,1089,7,912


In [48]:
mapp=geopandas.read_file('custom.geo.json')
mapp.crs = {'init': 'epsg:4326'}

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [24]:
mapp

,scalerank,featurecla,labelrank,sovereignt,sov_a3,adm0_dif,level,type,admin,adm0_a3,geou_dif,geounit,gu_a3,su_dif,subunit,su_a3,brk_diff,name,name_long,brk_a3,brk_name,brk_group,abbrev,postal,formal_en,formal_fr,note_adm0,note_brk,name_sort,name_alt,mapcolor7,mapcolor8,mapcolor9,mapcolor13,pop_est,gdp_md_est,pop_year,lastcensus,gdp_year,economy,income_grp,wikipedia,fips_10_,iso_a2,iso_a3,iso_n3,un_a3,wb_a2,wb_a3,woe_id,woe_id_eh,woe_note,adm0_a3_is,adm0_a3_us,adm0_a3_un,adm0_a3_wb,continent,region_un,subregion,region_wb,name_len,long_len,abbrev_len,tiny,homepart,filename,geometry
0,3,Admin-0 country,5,Netherlands,NL1,1,2,Country,Aruba,ABW,0,Aruba,ABW,0,Aruba,ABW,0,Aruba,Aruba,ABW,Aruba,None,Aruba,AW,Aruba,None,Neth.,None,Aruba,None,4,2,2,9,103065,2258.00,-99,2010,-99,6. Developing region,2. High income: nonOECD,-99,AA,AW,ABW,533,533,AW,ABW,23424736,23424736,Exact WOE match as country,ABW,ABW,-99,-99,North America,Americas,Caribbean,Latin America & Caribbean,5,5,5,4,-99,ABW.geojson,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,3,Admin-0 country,6,United Kingdom,GB1,1,2,Dependency,Anguilla,AIA,0,Anguilla,AIA,0,Anguilla,AIA,0,Anguilla,Anguilla,AIA,Anguilla,None,Ang.,AI,None,None,U.K.,None,Anguilla,None,6,6,6,3,14436,108.90,-99,-99,-99,6. Developing region,3. Upper middle income,-99,AV,AI,AIA,660,660,-99,-99,23424751,23424751,Exact WOE match as country,AIA,AIA,-99,-99,North America,Americas,Caribbean,Latin America & Caribbean,8,8,4,-99,-99,AIA.geojson,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
2,3,Admin-0 country,6,Antigua and Barbuda,ATG,0,2,Sovereign country,Antigua and Barbuda,ATG,0,Antigua and Barbuda,ATG,0,Antigua and Barbuda,ATG,0,Antigua and Barb.,Antigua and Barbuda,ATG,Antigua and Barb.,None,Ant.B.,AG,Antigua and Barbuda,None,None,None,Antigua and Barbuda,None,2,2,5,5,85632,1657.00,-99,2011,-99,6. Developing region,3. Upper middle income,-99,AC,AG,ATG,028,028,AG,ATG,23424737,23424737,Exact WOE match as country,ATG,ATG,-99,-99,North America,Americas,Caribbean,Latin America & Caribbean,17,19,6,4,1,ATG.geojson,"MULTIPOLYGON (((-61.77302 17.12653, -61.75642 ..."
3,6,Admin-0 country,8,Bajo Nuevo Bank (Petrel Is.),BJN,0,2,Indeterminate,Bajo Nuevo Bank (Petrel Is.),BJN,0,Bajo Nuevo Bank (Petrel Is.),BJN,0,Bajo Nuevo Bank (Petrel Is.),BJN,1,Bajo Nuevo Bank (Petrel Is.),Bajo Nuevo Bank (Petrel Islands),B41,Bajo Nuevo Bank (Petrel Is.),None,None,None,None,None,Disputed,"Claimed by Columbia, Jamaica, Nicaragua and th...",Bajo Nuevo Bank (Petrel Is.),None,3,7,6,-99,-99,-99.00,2012,-99,2012,7. Least developed region,5. Low income,-99,-99,-99,-99,-99,-099,-99,-99,-99,-99,No WOE equivalent.,-99,B41,-99,-99,North America,Americas,Caribbean,Latin America & Caribbean,28,32,0,-99,1,bajonuevobankpetrelis.geojson,"POLYGON ((-79.98929 15.79495, -79.98782 15.796..."
4,6,Admin-0 country,6,France,FR1,1,2,Dependency,Saint Barthelemy,BLM,0,Saint Barthelemy,BLM,0,Saint Barthelemy,BLM,0,St-Barthélemy,Saint-Barthélemy,BLM,St-Barthélemy,None,St. B.,BL,Saint-Barthélemy,None,Fr.,None,St-Barthélemy,None,7,5,9,11,7448,255.00,-99,-99,-99,2. Developed region: nonG7,1. High income: OECD,-99,TB,BL,BLM,652,652,-99,-99,56042304,56042304,Exact WOE match as country,BLM,BLM,-99,-99,North America,Americas,Caribbean,Latin America & Caribbean,13,16,6,4,-99,BLM.geojson,"POLYGON ((-62.83886 17.88198, -62.85094 17.890..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,5,Admin-0 country,4,Tonga,TON,0,2,Sovereign country,Tonga,TON,0,Tonga,TON,0,Tonga,TON,0,Tonga,Tonga,TON,Tonga,None,Tongo,TO,Kingdom of Tonga,None,None,None,Tonga,None,2,1,1,8,120898,549.00,-99,2006,-99,6. Developing region,4. Lower middle income,-99,TN,TO,TON,776,776,TO,TON,23424964,23424964,Exact WOE match as country,TON,TON,-99,-99,Oceania,Oceania,Polynesia,East A

In [0]:
mapp = mapp.rename(columns={'geometry': 'geometry','name':'Country_Region'}).set_geometry('geometry')

In [0]:
mapp.drop(mapp.columns.difference(['Country_Region','geometry']), 1, inplace=True)

In [51]:
mapp

,Country_Region,geometry
0,Aruba,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Anguilla,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
2,Antigua and Barb.,"MULTIPOLYGON (((-61.77302 17.12653, -61.75642 ..."
3,Bajo Nuevo Bank (Petrel Is.),"POLYGON ((-79.98929 15.79495, -79.98782 15.796..."
4,St-Barthélemy,"POLYGON ((-62.83886 17.88198, -62.85094 17.890..."
...,...,...
240,Tonga,"MULTIPOLYGON (((-176.20702 -22.33880, -176.210..."
241,Tuvalu,"MULTIPOLYGON (((179.90610 -9.41863, 179.90528 ..."
242,Vanuatu,"MULTIPOLYGON (((169.83815 -20.24863, 169.83009..."
243,Wallis and Futuna Is.,"MULTIPOLYGON (((-178.04674 -14.31260, -178.046..."


In [52]:
mapp.crs = {'init': 'epsg:4326'}

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [53]:
mapp.crs = {'init': 'epsg:4326'}

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [0]:
merged = pd.merge(mapp, df, on='Country_Region', how='left')

In [58]:
merged.head()

,Country_Region,geometry,Lat,Long_,Confirmed,Deaths,Recovered
0,Aruba,"POLYGON ((-69.99694 12.57758, -69.93639 12.531...",NaN,NaN,NaN,NaN,NaN
1,Anguilla,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ...",NaN,NaN,NaN,NaN,NaN
2,Antigua and Barb.,"MULTIPOLYGON (((-61.77302 17.12653, -61.75642 ...",NaN,NaN,NaN,NaN,NaN
3,Bajo Nuevo Bank (Petrel Is.),"POLYGON ((-79.98929 15.79495, -79.98782 15.796...",NaN,NaN,NaN,NaN,NaN
4,St-Barthélemy,"POLYGON ((-62.83886 17.88198, -62.85094 17.890...",NaN,NaN,NaN,NaN,NaN


In [0]:
values = {'Lat': 0, 'Long_': 0, 'Confirmed': 0,
              'Deaths': 0, 'Recovered': 0}
merged = merged.fillna(value=values)

In [61]:
merged

,Country_Region,geometry,Lat,Long_,Confirmed,Deaths,Recovered
0,Aruba,"POLYGON ((-69.99694 12.57758, -69.93639 12.531...",0.0,0.0,0.0,0.0,0.0
1,Anguilla,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ...",0.0,0.0,0.0,0.0,0.0
2,Antigua and Barb.,"MULTIPOLYGON (((-61.77302 17.12653, -61.75642 ...",0.0,0.0,0.0,0.0,0.0
3,Bajo Nuevo Bank (Petrel Is.),"POLYGON ((-79.98929 15.79495, -79.98782 15.796...",0.0,0.0,0.0,0.0,0.0
4,St-Barthélemy,"POLYGON ((-62.83886 17.88198, -62.85094 17.890...",0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
240,Tonga,"MULTIPOLYGON (((-176.20702 -22.33880, -176.210...",0.0,0.0,0.0,0.0,0.0
241,Tuvalu,"MULTIPOLYGON (((179.90610 -9.41863, 179.90528 ...",0.0,0.0,0.0,0.0,0.0
242,Vanuatu,"MULTIPOLYGON (((169.83815 -20.24863, 169.83009...",0.0,0.0,0.0,0.0,0.0
243,Wallis and Futuna Is.,"MULTIPOLYGON (((-178.04674 -14.31260, -178.046...",0.0,0.0,0.0,0.0,0.0


In [71]:
print (merged[78:90])

     Country_Region  ... Recovered
78  Siachen Glacier  ...       0.0
79            Japan  ...   14422.0
80       Kazakhstan  ...    5458.0
81       Kyrgyzstan  ...    1181.0
82         Cambodia  ...     123.0
83            Korea  ...       0.0
84           Kuwait  ...   12899.0
85          Lao PDR  ...       0.0
86          Lebanon  ...     715.0
87        Sri Lanka  ...     811.0
88            Macao  ...       0.0
89          Myanmar  ...       0.0

[12 rows x 7 columns]
